In [3]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import warnings
import geopy.distance

warnings.filterwarnings('ignore')

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

storage = "/Volumes/easystore/Drones/"

cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
per_second = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_details = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/gpx-with-census-data.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
locations = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/manually-label/sensitive-locations.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [9]:
locations['lat'] = locations['Coordinates'].apply(lambda x: x.split(',')[0].strip())
locations['lon'] = locations['Coordinates'].apply(lambda x: x.split(',')[1].strip())

flight_details = flight_details[flight_details['id'].isin(cv_flights['id'])].copy()

In [32]:
locations 

,Location,Address,Coordinates,lat,lon
0,Islamic Center of Chula Vista (Masjid Al Momin...,"1280 3rd Ave, Chula Vista, CA 91911","32.60769923913585, -117.06656860342576",32.60769923913585,-117.06656860342576
1,Islamic Center Of Eastlake,"821 Kuhn Dr # 108, Chula Vista, CA 91914","32.65398380177102, -116.9667393066541",32.65398380177102,-116.9667393066541
2,Immigration Services,"629 3rd Ave H, Chula Vista, CA 91910","32.630028919487536, -117.07458703364378",32.630028919487536,-117.07458703364378
3,Superior Immigration Services,"401 H St suite 9, Chula Vista, CA 91910","32.633602982231366, -117.08168888151182",32.633602982231366,-117.08168888151182
4,MA Immigration Services,"355 3rd. Avenue, Madrona St Suite 106, Chula V...","32.63954583155412, -117.07887788750898",32.63954583155412,-117.07887788750898
5,Victory Immigration Firm,"333 H St Suite 5000, Chula Vista, CA 91910","32.634980988389145, -117.07758831738357",32.634980988389145,-117.07758831738357
6,USCIS San Diego,"Sommerset Plaza, 1655 Broadway, Chula Vista, C...","32.597990028576646, -117.07634417330596",32.597990028576646,-117.07634417330596
7,Planned Parenthood - Chula Vista Health Center,"1295 Broadway #201, Chula Vista, CA 91911","32.60440650707405, -117.07651238265535",32.60440650707405,-117.07651238265535
8,Hilltop High,"555 Claire Ave, Chula Vista, CA 91910","32.638887017271585, -117.05609048595241",32.638887017271585,-117.05609048595241
9,vista square elementary,"540 G St, Chula Vista, CA 91910","32.63486911191133, -117.08348563114595",32.63486911191133,-117.08348563114595


In [33]:
def calculate_distance(row, target_lat, target_lon):
    origin = (float(row['latitude']), float(row['longitude']))
    target = (float(target_lat), float(target_lon))
    distance_in_meters = geopy.distance.geodesic(origin, target).meters
    return distance_in_meters * 3.28084  # Convert meters to feet
# Apply the function to each row in the DataFrame


In [68]:

def find_sensitive_flights(index):

    meta = {}
    flight_id = flight_ids.iloc[index]['id']
    flight = flight_details[flight_details['id'] == flight_id]
    meta['flight_id'] = flight_id
    for index, t_row in locations.iterrows():
        flight[f"close_to_{t_row['Location']}"] = False
        flight[f"distance_to_{t_row['Location']}"] = flight.apply(calculate_distance, axis=1, args=(t_row['lat'],t_row['lon']))
        
        flight.loc[flight[f"distance_to_{t_row['Location']}"]<=50,f"close_to_{t_row['Location']}"] = True
        
        if flight[flight[f"close_to_{t_row['Location']}"] == True].shape[0]>0:
            meta[f"close_to_{t_row['Location']}"] = True
        else:
            meta[f"close_to_{t_row['Location']}"] = False
    
    return  meta

In [73]:
flight_ids = flight_details[['id']].drop_duplicates()
with Pool(50) as pool:

    lingering_nd = list(
        tqdm(pool.imap(find_sensitive_flights, range(0, flight_ids.shape[0])), total=flight_ids.shape[0])
    )
    lingering_nd = pd.DataFrame(lingering_nd)

  0%|          | 0/8742 [00:00<?, ?it/s]

In [76]:
lingering_nd.to_csv('./sensitive-flights-do-not-delete.csv')

In [91]:
lingering_nd.columns.values

array(['flight_id',
       'close_to_Islamic Center of Chula Vista (Masjid Al Momineen) ',
       'close_to_Islamic Center Of Eastlake',
       'close_to_Immigration Services',
       'close_to_Superior Immigration Services ',
       'close_to_MA Immigration Services',
       'close_to_Victory Immigration Firm', 'close_to_USCIS San Diego ',
       'close_to_Planned Parenthood - Chula Vista Health Center',
       'close_to_Hilltop High', 'close_to_vista square elementary',
       'close_to_Mueller Charter School',
       'close_to_Bayfront Charter High School',
       'close_to_Chula Vista Learning Community Charter elementary school',
       'close_to_Chula Vista High School',
       'close_to_Harborside Elementary School',
       'close_to_Palomar High School',
       'close_to_Palomar Elementary School', 'close_to_Moneygram'],
      dtype=object)

In [93]:
lingering_nd[lingering_nd['close_to_USCIS San Diego ']==True]

,flight_id,close_to_Islamic Center of Chula Vista (Masjid Al Momineen),close_to_Islamic Center Of Eastlake,close_to_Immigration Services,close_to_Superior Immigration Services,close_to_MA Immigration Services,close_to_Victory Immigration Firm,close_to_USCIS San Diego,close_to_Planned Parenthood - Chula Vista Health Center,close_to_Hilltop High,close_to_vista square elementary,close_to_Mueller Charter School,close_to_Bayfront Charter High School,close_to_Chula Vista Learning Community Charter elementary school,close_to_Chula Vista High School,close_to_Harborside Elementary School,close_to_Palomar High School,close_to_Palomar Elementary School,close_to_Moneygram
753,b0b5140bf5ea70c3a6bf6677c5d6d3eb,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1610,cc14803ff81aae2f0c99267dfe830b50,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1957,25f501ceeb4bf448902cd23bf8674c1a,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2645,6ed52889f04873825d0ef89fcd7007c2,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3142,cf7c00f84cbedb2ff44a59abfb43b8b5,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3388,9cebf8afb0e6d4233e8eb00e9497456c,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
4557,67f4e17fa6a532e7c9f426a2a5ae7929,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
5041,f191e4f1346c354ed4db1c5a88f33556,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
6982,5fafbcee716c3bdb4e669e4a6740f8b5,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
7638,8eb583cfbae0e62cfc918d8603d1871c,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False


In [13]:
geopy.distance.geodesic((flight_details['latitude'].values[0],flight_details['longitude'].values[0]),(32.60769923913585,-117.06656860342576)).miles

2.4164226792951697

In [7]:
possible_cfs_by_distance = []
for _, flight in tqdm(orphan_flights.iterrows()):
#   find possible calls by time
    possible_cfs_for_flight = possible_calls_by_time_alone[possible_calls_by_time_alone['drone-id']==flight['id']]
    
    orphan_flight = orphan_flight_blocks[orphan_flight_blocks['id']==flight['id']].copy()
    orphan_flight['seconds'] = orphan_flight['seconds'].astype(int)

    lingering_blocks = orphan_flight[orphan_flight['seconds']>=60]
    lingering_blocks = cv_blocks[cv_blocks['GEOID20'].isin(lingering_blocks['GEOID20'])]
    if lingering_blocks.shape[0]>0:
        for _, cfs in possible_cfs_for_flight.iterrows():
            for _, block in lingering_blocks.iterrows():
                distance = geopy.distance.geodesic((cfs['Latitude'],cfs['Longitude']),(block['INTPTLAT20'],block['INTPTLON20'])).miles
                if str(cfs['Latitude'])=='0':
                    cfs["lingering-block"] = block['GEOID20']
                    cfs["distance"] = distance
                    possible_cfs_by_distance.append(cfs)

                elif distance <= .5:
                    cfs["lingering-block"] = block['GEOID20']
                    cfs["distance"] = distance
                    possible_cfs_by_distance.append(cfs)
                elif distance > 30:
                    cfs["lingering-block"] = block['GEOID20']
                    cfs["distance"] = distance
                    possible_cfs_by_distance.append(cfs)


,Location,Address,Coordinates,lat,lon
0,Islamic Center of Chula Vista (Masjid Al Momin...,"1280 3rd Ave, Chula Vista, CA 91911","32.60769923913585, -117.06656860342576",32.60769923913585,-117.06656860342576
1,Islamic Center Of Eastlake,"821 Kuhn Dr # 108, Chula Vista, CA 91914","32.65398380177102, -116.9667393066541",32.65398380177102,-116.9667393066541
2,Immigration Services,"629 3rd Ave H, Chula Vista, CA 91910","32.630028919487536, -117.07458703364378",32.630028919487536,-117.07458703364378
3,Superior Immigration Services,"401 H St suite 9, Chula Vista, CA 91910","32.633602982231366, -117.08168888151182",32.633602982231366,-117.08168888151182
4,MA Immigration Services,"355 3rd. Avenue, Madrona St Suite 106, Chula V...","32.63954583155412, -117.07887788750898",32.63954583155412,-117.07887788750898
5,Victory Immigration Firm,"333 H St Suite 5000, Chula Vista, CA 91910","32.634980988389145, -117.07758831738357",32.634980988389145,-117.07758831738357
6,USCIS San Diego,"Sommerset Plaza, 1655 Broadway, Chula Vista, C...","32.597990028576646, -117.07634417330596",32.597990028576646,-117.07634417330596
7,Planned Parenthood - Chula Vista Health Center,"1295 Broadway #201, Chula Vista, CA 91911","32.60440650707405, -117.07651238265535",32.60440650707405,-117.07651238265535
8,Hilltop High,"555 Claire Ave, Chula Vista, CA 91910","32.638887017271585, -117.05609048595241",32.638887017271585,-117.05609048595241
9,vista square elementary,"540 G St, Chula Vista, CA 91910","32.63486911191133, -117.08348563114595",32.63486911191133,-117.08348563114595


In [14]:
def build_paths(array_of_ids):
    compiled_flights =[] 
    for flight_id in array_of_ids:

        flight_path = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/{flight_id}.kml",driver='KML')
        flight_path['id'] = flight_id
        compiled_flights.append(flight_path)
    return pd.concat(compiled_flights)

In [15]:
geocodio_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/geocodio/all-flights-manifest_geocodio.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
# a = pd.read_csv('../../data/outputs/outputs_cv-flight-manifest.csv')

Loading data: 0it [00:00, ?it/s]

In [16]:
geocodio_flights[(geocodio_flights['address_map'].astype(str).str.contains('1700',case=False))&(geocodio_flights['address_map'].astype(str).str.contains('broadway',case=False))]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,...,Census Block Group,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code
1560,32.5930563443194,-117.074888979111,11:38am,58de5d94143d8527cc132af53a9aa87e,7-23-21,11:38am,1700 Broadway,NaN,Hot Team Assist,cvpd,...,1,060730132051017,06073013205,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
2836,32.61736464,-117.07162882,1:46pm,ed246d91644ba93bcf07893f9c27a44d,9-14-21,1:46pm,1700 Broadway,CVL77376,Subject Causing Disturbance,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
2837,32.6355209,-117.0731106,1:35pm,a9e795daf6e3f22f45aa3c98cd3f8315,9-14-21,1:35pm,1700 Broadway,CVL77376,Subject Causing Disturbance,cvpd,...,2,060730128002003,06073012800,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
3139,32.6174123,-117.07167632,8:16am,29b1f6e179d9564bfac69b9a187c7ab6,1-26-22,8:16am,1700 Broadway,CVL7019,DUI,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
6744,32.61742367,-117.07167327,2:44pm,9b0cdb6dbb05c3d58228ec011f4d59f1,9-27-22,2:44pm,1700 Broadway,CV2212713,Stolen vehicle,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
10147,32.61740644,-117.07166125,10:59am,057d62256c1263cc8c35d72250787527,7-13-23,10:59am,1300 Third Av / 1700 Broadway,L059083/ L059088,Disorderly Person/ Impaired Driver,cvpd,...,2,060730131022000,06073013102,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN


In [ ]:
# pool = 060730123021012


060730127003001

In [55]:
lede = per_flight_block_seconds[(per_flight_block_seconds['GEOID20']=='060730132063003')]
flights = []
for f in lede['id'].drop_duplicates().values:
    flight = per_flight_block_seconds[per_flight_block_seconds['id']==f]
    flight = flight[flight['GEOID20']=='060730127003001']
    if flight.shape[0]>0:
        flights.append(flight)

,id,GEOID20,seconds,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,NAME20,MTFCC20,UR20,...,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,HOUSING20,POP20,geometry,len
40868,5db1d2295546e6d0d2726877c1c081e2,060730127003001,50,6,73,12700,3001,Block 3001,G5040,U,...,U,S,40518,0,32.6365542,-117.0848466,81,239,"POLYGON ((-117.087049 32.636423, -117.08691 32...",15


In [18]:
pp_paths = build_paths(['29b1f6e179d9564bfac69b9a187c7ab6'])
pp_paths = pd.merge(pp_paths,geocodio_flights,how='left',on='id')

pp_paths[['id','geometry','lat_map','lon_map','type']].to_csv('./public_pool.csv',index=False)
# pp_paths[['geometry','id']].drop_duplicates().to_csv('./low-paths.csv',index=False)

In [43]:
flight_details[flight_details['id']=='4342f9e237e48c34ddd15271a0ed1dc9'][['sequence','latitude','longitude','altitude']].to_csv('./flight.csv',index=False)

In [61]:
pp_paths

,Name,Description,geometry,id,lat_map,lon_map,time,date,time_s,address_map,...,Census Block Group,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code
0,Home,,POINT Z (-117.08270 32.63997 22.14233),5db1d2295546e6d0d2726877c1c081e2,32.6022197,-117.0692582,4:03pm,7-02-21,4:03pm,Fourth Ave./ Orange Ave.,...,3,060730132063003,06073013206,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
1,Airdata.com,,"LINESTRING Z (-117.08270 32.63997 22.14233, -1...",5db1d2295546e6d0d2726877c1c081e2,32.6022197,-117.0692582,4:03pm,7-02-21,4:03pm,Fourth Ave./ Orange Ave.,...,3,060730132063003,06073013206,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN
